In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Define the initial types for the ONNX model
# Here, 'None' indicates that the first dimension is variable
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the trained model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the model to a file
with open("logistic_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [5]:

# Load the ONNX model and run inference
sess = rt.InferenceSession("logistic_regression.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

# Output the predictions from ONNX model
print(pred_onx)


[1 2 0 1 2 1 2 2 1 0 2 0 2 2 0 1 1 1 1 1 0 1 0 2 1 2 1 1 2 1]


In [6]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,pred_onx)) 

1.0
